In [9]:
suppressWarnings(
  suppressPackageStartupMessages(
    {
      library(readxl)
      library(writexl)
      library(dplyr)
      library(purrr)
      library(tibble)
      library(devtools)
    }
  )
)

# Installs from the local folder (your modified clone)
# if (requireNamespace("idiolect", quietly = TRUE)) {
#   message("Removing Installed Version of Idiolect")
#   invisible(capture.output(remove.packages("idiolect")))
# }

message("Installing CRAN Version of Idiolect")
devtools::install("/Users/user/Documents/GitHub/idiolect", upgrade = "never")

library(idiolect)

Installing CRAN Version of Idiolect
── R CMD build ─────────────────────────────────────────────────────────────────
✔  checking for file ‘/Users/user/Documents/GitHub/idiolect/DESCRIPTION’ ...
─  preparing ‘idiolect’:
✔  checking DESCRIPTION meta-information ...
─  checking for LF line-endings in source and make files and shell scripts
─  checking for empty or unneeded directories
─  building ‘idiolect_1.1.1.9000.tar.gz’
   
Running /Library/Frameworks/R.framework/Resources/bin/R CMD INSTALL \
  /var/folders/xx/hy496x3x5sn4hy9gy1fk19lw0000gp/T//Rtmp8xNGuY/idiolect_1.1.1.9000.tar.gz \
  --install-tests 
* installing to library ‘/Library/Frameworks/R.framework/Versions/4.2-arm64/Resources/library’
* installing *source* package ‘idiolect’ ...
** using staged installation
** R
** data
*** moving datasets to lazyload DB
** inst
** tests
** byte-compile and prepare package for lazy loading
Warning messages:
1: package ‘quanteda’ was built under R version 4.2.3 
2: In .recacheSubclasses(def@

In [10]:
base_loc <- '/Volumes/BCross/av_datasets_experiments/Baseline Results'

data <- readRDS(paste0(base_loc, "/lambdag_results.rds"))

In [11]:
distinct_datasets <- data |>
  select(data_type, corpus) |>
  filter(corpus != "ACL") |>
  distinct()

In [12]:
distinct_datasets

  data_type            corpus
1      test              Wiki
2      test             Enron
3      test Perverted Justice
4      test     StackExchange
5  training              Wiki
6  training             Enron
7  training Perverted Justice
8  training     StackExchange

In [13]:
test_distinct <- distinct_datasets |> filter(data_type == 'test')
training_distinct <- distinct_datasets |> filter(data_type == 'training')

In [14]:
calculate_single_distinct <- function(score_df, problem_df){

  results_list <- vector("list", nrow(problem_df))
  n_total <- nrow(problem_df)

  for (i in seq_len(n_total)) {
    message("Processing row ", i, " out of ", n_total)

    filtered_problem <- problem_df[i, ]

    d_type <- filtered_problem |> dplyr::pull(data_type) |> as.character()
    corp   <- filtered_problem |> dplyr::pull(corpus) |> as.character()
    print(corp)
    filtered_score <- score_df |>
      dplyr::filter(
        data_type == d_type,
        corpus == corp
      )

    if (nrow(filtered_score) == 0) {
      next
    }

    res <- performance(filtered_score)
    perf <- res$evaluation

    perf <- perf |>
      dplyr::mutate(
        data_type = d_type,
        corpus = corp
      ) |>
      dplyr::relocate(data_type, corpus)

    results_list[[i]] <- perf
  }

  dplyr::bind_rows(results_list)
}

In [15]:
training_results <- calculate_single_distinct(data, training_distinct)
saveRDS(training_results, paste0(base_loc, "/training/lambdag_performance.rds"))

Processing row 1 out of 4
[1] "Wiki"
  |======================================================================| 100%
Processing row 2 out of 4
[1] "Enron"
  |======================================================================| 100%
Processing row 3 out of 4
[1] "Perverted Justice"
  |======================================================================| 100%
Processing row 4 out of 4
[1] "StackExchange"
  |======================================================================| 100%


In [16]:
test_results <- calculate_single_distinct(data, test_distinct)
saveRDS(test_results, paste0(base_loc, "/test/lambdag_performance.rds"))

Processing row 1 out of 4
[1] "Wiki"
  |======================================================================| 100%
Processing row 2 out of 4
[1] "Enron"
  |======================================================================| 100%
Processing row 3 out of 4
[1] "Perverted Justice"
  |======================================================================| 100%
Processing row 4 out of 4
[1] "StackExchange"
  |======================================================================| 100%


In [20]:
test_results

  data_type            corpus      Cllr  Cllr_min      EER Mean TRUE LLR
1      test              Wiki 0.6467854 0.6004004 22.80702      2.907301
2      test             Enron 0.5425406 0.3909434 14.66667      6.098797
3      test Perverted Justice 0.6330302 0.5445367 19.62025      1.699945
4      test     StackExchange 0.7207091 0.6011692 20.68966      1.120950
  Mean FALSE LLR TRUE trials FALSE trials       AUC Balanced Accuracy Precision
1      -1.375433         114          114 0.8573023         0.7857143 0.8555556
2      -1.953219          50           50 0.9240451         0.8645833 0.9069767
3      -2.344277         158          158 0.8796433         0.7980769 0.7569061
4      -1.387573         116          116 0.8636504         0.7938596 0.7596899
     Recall        F1  TP FN FP  TN
1 0.6875000 0.7623762  77 35 13  99
2 0.8125000 0.8571429  39  9  4  44
3 0.8782051 0.8130564 137 19 44 112
4 0.8596491 0.8065844  98 16 31  83

In [17]:
calculate_calibrated_distinct <- function(score_df, problem_df){

  training_data <- score_df |> filter(data_type == 'training')
  test_data <- score_df |> filter(data_type == 'test')

  training_problems <- problem_df |> filter(data_type == 'training') |> select(-data_type)
  test_problems <- problem_df |> filter(data_type == 'test') |> select(-data_type)

  combined_problems <- test_problems |>
    inner_join(training_problems, by=c('corpus'))

  results_list <- vector("list", nrow(combined_problems))
  n_total <- nrow(combined_problems)

  for (i in seq_len(n_total)) {
    message("Processing row ", i, " out of ", n_total)

    filtered_problem <- problem_df[i, ]

    corp   <- filtered_problem |> dplyr::pull(corpus) |> as.character()

    filtered_training <- training_data |>
      dplyr::filter(
        corpus == corp
      )

    filtered_test <- test_data |>
      dplyr::filter(
        corpus == corp
      )

    res <- performance(filtered_test, training = filtered_training)
    perf <- res$evaluation

    perf <- perf |>
      dplyr::mutate(
        corpus = corp
      ) |>
      dplyr::relocate(corpus)

    results_list[[i]] <- perf
  }

  dplyr::bind_rows(results_list)
}

In [18]:
combined_data <- calculate_calibrated_distinct(data, distinct_datasets)
saveRDS(combined_data, paste0(base_loc, "/lambdag_calibrated_performance.rds"))

Processing row 1 out of 4
Processing row 2 out of 4
Processing row 3 out of 4
Processing row 4 out of 4


In [19]:
readRDS(paste0(base_loc, '/lambdag_calibrated_performance.rds'))

             corpus      Cllr  Cllr_min      EER Mean TRUE LLR Mean FALSE LLR
1              Wiki 0.6550200 0.5963798 22.80702     4.1715712     -1.9486297
2             Enron 0.4954981 0.3736823 14.00000     5.9337448     -2.0219819
3 Perverted Justice 0.6628230 0.5408757 19.62025     2.3732317     -4.2030882
4     StackExchange 0.7828296 0.5892073 20.68966     0.5049322     -0.4385229
  TRUE trials FALSE trials       AUC Balanced Accuracy Precision    Recall
1         114          114 0.8616470         0.7812500 0.8461538 0.6875000
2          50           50 0.9294705         0.8750000 0.9285714 0.8125000
3         158          158 0.8934706         0.7852564 0.9009009 0.6410256
4         116          116 0.8671899         0.7938596 0.7410072 0.9035088
         F1  TP FN FP  TN
1 0.7586207  77 35 14  98
2 0.8666667  39  9  3  45
3 0.7490637 100 56 11 145
4 0.8142292 103 11 36  78